# Second Data Cleaning 

In [5]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

In [3]:
df1 = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Cleaning Data\First Cleaning\df1_unique_streets_only.csv")  
df2 = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Cleaning Data\First Cleaning\df2_unique_streets_zipcode.csv")  

In [6]:
replacements = {
    "ROAD": "RD", "RD": "RD",
    "STREET": "ST", "ST": "ST",
    "AVENUE": "AVE", "AVE": "AVE",
    "DRIVE": "DR", "DR": "DR",
    "COURT": "CT", "CT": "CT",
    "LANE": "LN", "LN": "LN",
    "CIRCLE": "CIR", "CIR": "CIR",
    "TRAIL": "TRL", "TRL": "TRL",
    "PARKWAY": "PKWY", "PKWY": "PKWY",
    "HIGHWAY": "HWY", "HWY": "HWY",
    "PLACE": "PL", "PL": "PL",
    "PATH": "PATH",
    "WAY": "WAY",
    "LOOP": "LOOP",
    "COVE": "CV", "CV": "CV",
    "NORTH": "N", "N": "N",
    "SOUTH": "S", "S": "S",
    "EAST": "E", "E": "E",
    "WEST": "W", "W": "W",
    "NORTHEAST": "NE", "NE": "NE",
    "NORTHWEST": "NW", "NW": "NW",
    "SOUTHEAST": "SE", "SE": "SE",
    "SOUTHWEST": "SW", "SW": "SW",
    "NB": "NB", "SB": "SB", "EB": "EB", "WB": "WB"
}

direction_garbage = {"BLOCK", "NB", "SB", "EB", "WB", "NORTHBOUND", "SOUTHBOUND", "EASTBOUND", "WESTBOUND"}

ordinals = {
    "FIRST": "1ST", "SECOND": "2ND", "THIRD": "3RD", "FOURTH": "4TH",
    "FIFTH": "5TH", "SIXTH": "6TH", "SEVENTH": "7TH", "EIGHTH": "8TH",
    "NINTH": "9TH", "TENTH": "10TH"
}

def advanced_clean(s):
    if pd.isna(s) or s == 'nan':
        return ''
        
    s = s.upper()
    s = re.sub(r"^\d+\s+", "", s)
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\d+ BLOCK", "", s)
    s = re.sub(r"BLOCK", "", s)
    s = re.sub(r"\d+", "", s)
    s = re.sub(r"1/2|1 2", "", s)
    s = re.sub(r"SERVICE RD|SVRD|FWY|EXPY|EXPRESSWAY|IH|US|SH|FM", "", s)
    s = re.sub(r"\s+", " ", s)

    tokens = s.split()
    clean_tokens = []

    for t in tokens:
        if t in direction_garbage:
            continue
        t = ordinals.get(t, t)
        t = replacements.get(t, t)
        clean_tokens.append(t)

    result = " ".join(clean_tokens).strip()

    # 🔴 Drop meaningless outputs (only directions or too short)
    bad = {"N", "S", "E", "W", "NE", "NW", "SE", "SW"}
    if result in bad or len(result) < 3:
        return ""

    return result

## Second Match Rate

In [7]:
df1['street_clean1'] = df1['street_clean1'].apply(advanced_clean)
df1_unique = df1.drop_duplicates(subset=["street_clean1"])

In [8]:
df2['street_clean2'] = df2['street_clean2'].apply(advanced_clean)
df2_unique = df2.drop_duplicates(subset=["street_clean2", "POSTCODE"])

In [9]:
df1_unique.to_csv('df1_cleaned_final.csv', index=False)
df2_unique.to_csv('df2_cleaned_final.csv', index=False)

In [10]:
common = set(df1_unique["street_clean1"]) & set(df2_unique["street_clean2"])
common.discard('')
match_rate = (len(common) / len(df1_unique["street_clean1"])) * 100 if len(df1_unique) > 0 else 0

In [11]:
print("Match rate :", match_rate, "%")
print("number of common streets:", len(common))
print("common examples:", list(common)[:10])

Match rate : 40.5235828645608 %
number of common streets: 1873
common examples: ['ELLINGTON CIR', 'CABANA LN', 'TAHOE TRL', 'FIREOAK DR', 'REID DR', 'MC KINLEY AVE', 'SOUTHPORT DR', 'OAKWOOD DR', 'PALO BLANCO LN', 'ROCKWOOD LN']


## Second Integrate

In [14]:
crimes = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Datasets\Municipal_Court_Caseload_Information_FY_2023.csv")
df2 = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Cleaning Data\Second Cleaning\df2_cleaned_final.csv")  # cleaned Street-ZIP
neigh = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Datasets\austin_zip_to_neighborhood_scraped.csv")
pop = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Datasets\austin_population_by_zip_scraped.csv")

In [15]:
crimes['street_clean'] = crimes['Offense Street Name'].apply(advanced_clean)

In [16]:
crimes_with_zip = pd.merge(crimes, df2, left_on='street_clean', right_on='street_clean2', how='left')

In [17]:
crimes_with_neigh = pd.merge(crimes_with_zip, neigh, left_on='POSTCODE', right_on='ZIP_Code', how='left')

In [18]:
valid_pop = pop[pop['Population'] > 0]
crimes_integrated = pd.merge(crimes_with_neigh, valid_pop[['ZIP_Code', 'Population']], left_on='POSTCODE', right_on='ZIP_Code', how='left')

In [19]:
analysis = crimes_integrated.groupby('Neighborhood').agg({
    'Offense Case Type': 'count',
    'Population': 'mean'
}).reset_index()
analysis.rename(columns={'Offense Case Type': 'Crime_Count'}, inplace=True)
analysis['Crime_Rate_per_1000'] = (analysis['Crime_Count'] / analysis['Population']) * 1000
analysis = analysis.sort_values('Crime_Rate_per_1000', ascending=False)

In [20]:
crimes_integrated.to_csv('final_integrated_all.csv', index=False)
analysis.to_csv('analysis_neighborhood_rate.csv', index=False)

In [21]:
print("Match rate:", (crimes_integrated['POSTCODE'].notna().mean() * 100))
print(analysis.head(10))

Match rate: 93.93981111814753
                                         Neighborhood  Crime_Count  \
6                                     Downtown Austin       105399   
3                                          Cherrywood        31475   
11                                          Hyde Park        61331   
23                                           Rosedale        30070   
8                                         East Austin        66064   
17                        North University, Hyde Park        94056   
29                             Tarrytown, Clarksville        37075   
18                                    Northwest Hills        28713   
14                              Mueller, Windsor Park        35259   
26  South Lamar, Zilker, Bouldin Creek, Travis Hei...        35315   

      Population  Crime_Rate_per_1000  
6   11625.000000          9066.580645  
3    6618.000000          4755.968571  
11  17071.000000          3592.701072  
23   8426.000000          3568.715879  

## Persisting the Integrated Data in a DBMS

In [22]:
import sqlite3

In [24]:
crimes_integrated = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Integration_Final\final_integrated_all.csv")

C:\Users\broba\AppData\Local\Temp\ipykernel_16712\2198711564.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crimes_integrated = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Project\Integration_Final\final_integrated_all.csv")


In [25]:
conn = sqlite3.connect('austin_crimes_project.db')
crimes_integrated.to_sql('crimes_data', conn, if_exists='replace', index=False)

726380